In [ ]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import DonutProcessor, VisionEncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa").to(device)

In [3]:
x_dict = {
    "219gbqQt+ML.jpg": "height",
    "218vf17tHkL.jpg": "weight",
    "21-VzxP3BDL.jpg": "item_volume",
    "217V+UhIrHL.jpg": "length",
    "11j0F4QOiFL.jpg": "height",
    "211sXYcOHcL.jpg": "height",
    "218zo3iJ2IL.jpg": "length",
    "213VIsNlvzL.jpg": "height",
    "21+quvMwZSL.jpg": "weight",
    "217+y-mckBL.jpg": "weight",
    "211EIgVhPEL.jpg": "voltage",
    "218tBdpDGPS.jpg": "length",
    "21-V2Kx5BVL.jpg": "length"
}

y_dict = {
    "219gbqQt+ML.jpg": "12 cm",
    "218vf17tHkL.jpg": "250 mg",
    "21-VzxP3BDL.jpg": "200 ml",
    "217V+UhIrHL.jpg": "5 cm",
    "11j0F4QOiFL.jpg": "2.75 inches",
    "211sXYcOHcL.jpg": "8 cm",
    "218zo3iJ2IL.jpg": "44.2 cm",
    "213VIsNlvzL.jpg": "11 cm",
    "21+quvMwZSL.jpg": "1.6 lbs",
    "217+y-mckBL.jpg": "400 mg",
    "211EIgVhPEL.jpg": "3.7 V",
    "218tBdpDGPS.jpg": "104.5 inches",
    "21-V2Kx5BVL.jpg": "80 inches"
}


In [1]:
training_args = Seq2SeqTrainingArguments(
    # predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    # logging_steps=100,
    save_steps=500,
    eval_steps=500,
    save_total_limit=3,
    num_train_epochs=3,
    output_dir="./donut-finetuned-docvqa",
    fp16=True,
)

NameError: name 'Seq2SeqTrainingArguments' is not defined

In [ ]:

# class ImageDataset(Dataset):
#     def __init__(self, image_dir, x_dict, y_dict, transform=None):
#         self.image_dir = image_dir
#         self.transform = transform
#         self.images = list(x_dict.keys())
#         self.input_ids = list(x_dict.values())
#         self.labels = list(y_dict.values())
#         self.pre_finetune_text = '''Given the image, what is the''' # to stay consistent for finetuning
#         self.image_files = os.listdir(image_dir)
#         print(len(self.images), len(self.input_ids), len(self.labels), len(self.image_files))
#         assert type(self.labels) == list, "Answer should be a list of strings"
        

#     def __len__(self):
#         return len(self.input_ids)

#     def __getitem__(self, idx):
#         img_name = os.path.join(self.image_dir, self.image_files[idx])
#         image = Image.open(img_name)
#         if self.transform:
#             image = self.transform(image)
#         return image


# # Create dataset
# dataset = ImageDataset("/home/arjun/Desktop/Github/AmazonML-Hackathon/images/test", x_dict, y_dict, processor)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    # predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    # logging_steps=100,
    save_steps=500,
    eval_steps=500,
    save_total_limit=3,
    num_train_epochs=3,
    output_dir="./donut-finetuned-docvqa",
    fp16=True,
)

# # Define data collator
# def collate_fn(batch):
#     print('started colating')
#     input_ids = torch.stack([item['input_ids'] for item in batch])
#     attention_mask = torch.stack([item['attention_mask'] for item in batch])
#     labels = torch.stack([item['labels'] for item in batch])
#     pixel_values = torch.stack([item['pixel_values'] for item in batch])
#     print(input_ids.shape, attention_mask.shape, labels.shape, pixel_values.shape)
    
#     return {
#         'input_ids': input_ids,
#         'attention_mask': attention_mask,
#         'labels': labels,
#         'pixel_values': pixel_values,}


In [ ]:
# class ImageDataset(Dataset):
#     def __init__(self, image_dir, x_dict, y_dict, processor):
#         self.image_dir = image_dir
#         self.processor = processor
#         self.images = list(x_dict.keys())
#         self.input_ids = list(x_dict.values())
#         self.labels = list(y_dict.values())
#         self.pre_finetune_text = 'Given the image, what is the'
#         self.image_files = os.listdir(image_dir)
#         print(len(self.images), len(self.input_ids), len(self.labels), len(self.image_files))
#         assert type(self.labels) == list, "Answer should be a list of strings"
        
#     def __len__(self):
#         return len(self.input_ids)

#     def __getitem__(self, idx):
#         img_name = os.path.join(self.image_dir, self.image_files[idx])
#         image = Image.open(img_name).convert("RGB")
        
#         question = f"{self.pre_finetune_text} {self.input_ids[idx]}?"
#         answer = self.labels[idx]
        
#         encoding = self.processor(images=image, text=question, return_tensors="pt")
        
#         # Add labels to the encoding
#         encoding['labels'] = self.processor.tokenizer(answer, return_tensors="pt").input_ids
        
#         # Remove the batch dimension
#         for k,v in encoding.items():
#             encoding[k] = v.squeeze()
        
#         return encoding

# def collate_fn(batch):
#     print('started collating')
#     return processor.pad(batch, padding=True, return_tensors="pt")

# # Create dataset
# dataset = ImageDataset("/home/arjun/Desktop/Github/AmazonML-Hackathon/images/test", x_dict, y_dict, processor)

# # Define trainer
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset,
#     tokenizer=processor.tokenizer,
#     data_collator=collate_fn,
)

In [ ]:
# import torch
# from PIL import Image
# from torch.utils.data import Dataset
# import os

# class ImageDataset(Dataset):
#     def __init__(self, image_dir, x_dict, y_dict, processor):
#         self.image_dir = image_dir
#         self.processor = processor
#         self.images = list(x_dict.keys())
#         self.questions = list(x_dict.values())
#         self.answers = list(y_dict.values())
#         self.pre_finetune_text = 'Given the image, what is the'
#         self.image_files = os.listdir(image_dir)
#         print(len(self.images), len(self.questions), len(self.answers), len(self.image_files))
#         assert type(self.answers) == list, "Answer should be a list of strings"
        
#     def __len__(self):
#         return len(self.questions)

#     def __getitem__(self, idx):
#         img_name = os.path.join(self.image_dir, self.image_files[idx])
#         image = Image.open(img_name).convert("RGB")
        
#         question = f"{self.pre_finetune_text} {self.questions[idx]}?"
#         answer = self.answers[idx]
        
#         # Process the image
#         pixel_values = self.processor(images=image, return_tensors="pt").pixel_values.squeeze()
        
#         # Tokenize the question and answer
#         question_encoding = self.processor.tokenizer(question, padding="max_length", max_length=512, truncation=True, return_tensors="pt")
#         answer_encoding = self.processor.tokenizer(answer, padding="max_length", max_length=512, truncation=True, return_tensors="pt")
        
#         return {
#             "pixel_values": pixel_values,
#             "input_ids": question_encoding.input_ids.squeeze(),
#             "attention_mask": question_encoding.attention_mask.squeeze(),
#             "labels": answer_encoding.input_ids.squeeze()
#         }

# def collate_fn(batch):
#     print('started collating')
    
#     # Initialize dictionaries to store batched data
#     batched_data = {
#         'pixel_values': [],
#         'input_ids': [],
#         'attention_mask': [],
#         'labels': []
#     }

#     # Collect data from each item in the batch
#     for item in batch:
#         batched_data['pixel_values'].append(item['pixel_values'])
#         batched_data['input_ids'].append(item['input_ids'])
#         batched_data['attention_mask'].append(item['attention_mask'])
#         batched_data['labels'].append(item['labels'])

#     # Convert lists to tensors and pad where necessary
#     batched_data['pixel_values'] = torch.stack(batched_data['pixel_values'])
#     batched_data['input_ids'] = torch.nn.utils.rnn.pad_sequence(batched_data['input_ids'], batch_first=True, padding_value=processor.tokenizer.pad_token_id)
#     batched_data['attention_mask'] = torch.nn.utils.rnn.pad_sequence(batched_data['attention_mask'], batch_first=True, padding_value=0)
#     batched_data['labels'] = torch.nn.utils.rnn.pad_sequence(batched_data['labels'], batch_first=True, padding_value=-100)  # -100 is often used for ignored label indices

#     return batched_data

# # Create dataset
# dataset = ImageDataset("/home/arjun/Desktop/Github/AmazonML-Hackathon/images/test", x_dict, y_dict, processor)

# # Define trainer
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset,
#     tokenizer=processor.tokenizer,
#     data_collator=collate_fn,
# )

In [ ]:
import torch
from PIL import Image
from torch.utils.data import Dataset
import os

class ImageDataset(Dataset):
    def __init__(self, image_dir, x_dict, y_dict, processor):
        self.image_dir = image_dir
        self.processor = processor
        self.images = list(x_dict.keys())
        self.questions = list(x_dict.values())
        self.answers = list(y_dict.values())
        self.pre_finetune_text = 'Given the image, what is the'
        self.image_files = os.listdir(image_dir)
        print(len(self.images), len(self.questions), len(self.answers), len(self.image_files))
        assert type(self.answers) == list, "Answer should be a list of strings"
        
    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_name).convert("RGB")
        
        question = f"{self.pre_finetune_text} {self.questions[idx]}?"
        answer = self.answers[idx]
        
        # Prepare the inputs for the Donut model
        encoding = self.processor(images=image, text=question, return_tensors="pt")
        
        # Add the answer as the target text
        encoding["labels"] = self.processor.tokenizer(answer, return_tensors="pt").input_ids
        
        # Remove batch dimension
        for k,v in encoding.items():
            encoding[k] = v.squeeze()
        
        return encoding

def collate_fn(batch):
    print('started collating')
    
    # Initialize dictionaries to store batched data
    batched_data = {
        'pixel_values': [],
        'labels': []
    }

    max_label_length = max(item['labels'].size(0) for item in batch)

    # Collect data from each item in the batch
    for item in batch:
        batched_data['pixel_values'].append(item['pixel_values'])
        
        # Pad labels to max length in batch
        labels = item['labels']
        padded_labels = torch.full((max_label_length,), -100, dtype=torch.long)
        padded_labels[:labels.size(0)] = labels
        batched_data['labels'].append(padded_labels)

    # Stack tensors
    batched_data['pixel_values'] = torch.stack(batched_data['pixel_values'])
    batched_data['labels'] = torch.stack(batched_data['labels'])

    return batched_data

# Create dataset
dataset = ImageDataset("/home/arjun/Desktop/Github/AmazonML-Hackathon/images/test", x_dict, y_dict, processor)

# Define trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=processor.tokenizer,
    data_collator=collate_fn,
)

In [8]:
# Define trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=processor.tokenizer,
    data_collator=collate_fn,
)


In [ ]:
trainer.train()


# Inspection

In [ ]:

dataset[1]